In [1]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

packageName='07-sklearn.ensemble.a-bagging'
classifierName='RandomForestClassifier'
extraParameterName='7-SmoteOverSampling'

print('done')

done


In [2]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

done


In [3]:
import pandas as pd

dfTrx0 = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
predictors = fd.getPredictors(dfTrx0)
dfTrx=fd.getStandardScaledData('export'+date+'.csv',source,predictors)
dfTrx.head(5)

,mcd_fraud_score,vaa_score,TRX_3D_SECURED,trx_accepted,trx_cnp,ecom_indicator_group,ecom,trx_authentication_group,ch_present_group,trx_response_code_group,...,clusterCardHolder_cluster4,clusterMerchant_M0,clusterMerchant_M1,clusterMerchant_M2,clusterMerchant_M3,nbPreviousTrx,previousTrxAmountSumLog,trx_amount_log10,distancePrevTrx,Class
0,5.047982e-01,9.678365e-16,-0.316345,0.410697,0.895963,1.188319,0.656144,-0.640003,0.725223,-0.119809,...,-0.538119,1.055837,-0.358732,-0.224417,-0.721017,-0.445922,-0.731315,0.639959,0.409416,0
1,-6.112106e-01,9.678365e-16,3.161103,0.410697,0.895963,-0.841525,0.656144,1.562492,0.725223,-0.119809,...,-0.538119,-0.947116,-0.358732,-0.224417,1.386930,-0.445922,-0.731315,0.028804,-1.283070,0
2,1.090505e+00,9.678365e-16,-0.316345,0.410697,0.895963,1.188319,0.656144,-0.640003,0.725223,-0.119809,...,-0.538119,1.055837,-0.358732,-0.224417,-0.721017,-0.445922,-0.731315,-0.575760,0.409416,0
3,-6.748697e-16,9.678365e-16,-0.316345,0.410697,-1.116118,-0.841525,-1.524055,1.562492,-1.378886,-0.119809,...,-0.538119,-0.947116,-0.358732,-0.224417,-0.721017,-0.445922,-0.731315,0.358182,-0.100479,0
4,-6.748697e-16,5.096885e-01,-0.316345,0.410697,-1.116118,-0.841525,0.656144,-0.640003,0.725223,-0.119809,...,1.858325,1.055837,-0.358732,-0.224417,-0.721017,-0.445922,-0.731315,0.333060,0.409416,0


In [4]:
# Hyperparameters Results

n_estimatorsFound=63
max_depthFound=30

resultMd.update_hyperparameter_config_result(packageName,classifierName,extraParameterName,max_depthFound,n_estimatorsFound)


print('done')

done


In [5]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx0)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train, y_train = sp.smoteOverSampling(x_train0, y_train0,rateOverSampling=5)

Before Sampling shape (847879, 39) fraud rate 0.00123 
Duration 1.7 s 
After Sampling shape  (852039, 39) fraud rate 0.00614 


# Hyperparameters tuning

In [7]:
%%script false

from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier

modelClf = RandomForestClassifier(random_state=42)
dic_param={
    'n_estimators': randint(35,90),
    'max_depth': randint(17,33)
}
res=fd.hyperparameterSelectionRandomizedSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train, iter=4)
print(res)

#{'max_depth': 30, 'n_estimators': 67}
#0.2327854097160203
#score   0.9789731051344743



Couldn't find program: 'false'


In [ ]:
#%%script false
from sklearn.ensemble import RandomForestClassifier

modelClf = RandomForestClassifier(random_state=42)

dic_param={
    'n_estimators':[61,62,63,64],
    'max_depth':[28,30,32]
}
res=fd.hyperparameterSelectionGridSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)


#{'max_depth': 30, 'n_estimators': 65}
#0.8422744671993494
#scoref1 0.9903362968689602

#{'max_depth': 30, 'n_estimators': 63}
#0.843754587153043
#scoref1 0.9881874515879163



Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START max_depth=28, n_estimators=61..............................
[CV 1/5; 1/12] END max_depth=28, n_estimators=61;, score=0.518 total time= 2.4min
[CV 2/5; 1/12] START max_depth=28, n_estimators=61..............................
[CV 2/5; 1/12] END max_depth=28, n_estimators=61;, score=0.923 total time= 2.5min
[CV 3/5; 1/12] START max_depth=28, n_estimators=61..............................
[CV 3/5; 1/12] END max_depth=28, n_estimators=61;, score=0.922 total time= 2.5min
[CV 4/5; 1/12] START max_depth=28, n_estimators=61..............................
[CV 4/5; 1/12] END max_depth=28, n_estimators=61;, score=0.925 total time= 2.5min
[CV 5/5; 1/12] START max_depth=28, n_estimators=61..............................
[CV 5/5; 1/12] END max_depth=28, n_estimators=61;, score=0.918 total time= 2.5min
[CV 1/5; 2/12] START max_depth=28, n_estimators=62..............................
[CV 1/5; 2/12] END max_depth=28, n_estimato

# Final validation

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

 
modelClf = RandomForestClassifier(random_state=42)
parameters={'max_depth': max_depthFound, 'n_estimators':n_estimatorsFound}
modelClf.set_params(**parameters)
modelClf.fit(x_train, y_train)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

f1Train=fd.print_scores(y_train, predsTrain,'f1', False)
f1Test=fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)

diffF1=f1Train-f1Test
print("diffF1",diffF1)

In [ ]:
files = fd.getAllFiles()
predictors = fd.getPredictors(dfTrx0)

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx=fd.getStandardScaledData(file,source,predictors)
    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result(packageName,classifierName,extraParameterName, f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3],diffF1)

